In [1]:
# En caso de no tenerlo
# % pip install mlflow

In [2]:
import mlflow
import mlflow.pyfunc
import pandas as pd
import joblib

Obteniendo los datos simulados, hechos sinteticamente para este prueba

In [15]:
df = pd.read_csv('/home/project/data/simulados/simulados.csv')
df = df.drop(['TIPO_PACIENTE',], axis=1)

Se usan los mismos encoder y scaler que se usaron para los datos de entrenamiento

In [16]:
scaler = joblib.load('/home/project/models/preprocess_utils/scaler.pkl')
encoder = joblib.load('/home/project/models/preprocess_utils/encoder.pkl')

In [17]:
encoded_columns  = encoder.transform(df[['SEXO', 'ESTATUS_CASO', 'RESULTADO_PCR']])
scaled_columns = scaler.fit_transform(df.drop(['SEXO', 'ESTATUS_CASO', 'RESULTADO_PCR'], axis=1))

In [18]:
encoded_scaled_df = pd.DataFrame(encoded_columns, columns=encoder.get_feature_names_out(['SEXO', 'ESTATUS_CASO', 'RESULTADO_PCR']))
scaled_df = pd.DataFrame(scaled_columns, columns=df.drop(['SEXO', 'ESTATUS_CASO', 'RESULTADO_PCR'], axis=1).columns)
data = pd.concat([encoded_scaled_df, scaled_df], axis=1)

Configurar MLFlow para acceder al servidor de Dagshub que contiene los modelos

In [7]:
mlflow.set_tracking_uri("https://dagshub.com/empleo-inducido/project-ml.mlflow")

Dentro de variables `model_name` y `model_version` se guardan como estan etiquedos en MLFLow.

In [8]:
model_name = "testing-model"
model_version = 'latest'

model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")

/usr/local/lib/python3.10/site-packages/mlflow/store/artifact/utils/models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])


Se hace la predicción con los datos de muestra.

In [19]:
model.predict(data)

array([1, 1, 1, 1, 1, 1])